# Introduction

Python의 Model I/O 에 대해 학습함

# FewShotPromptTempalte

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts.few_shot import FewShotPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts import PromptTemplate

# Shot :

chat = ChatOpenAI(
    model = "gpt-4.1-nano",
    temperature = 0.1,
    streaming = True,
    callbacks = [
        StreamingStdOutCallbackHandler()
    ],
)

examples = [
    {
        "question": "What do you know about France?",
        "answer": """
        Here is what I know:
        Capital: Paris
        Language: French
        Food: Wine and Cheese
        Currency: Euro
        """,
    },
    {
        "question": "What do you know about Italy?",
        "answer": """
        I know this:
        Capital: Rome
        Language: Italian
        Food: Pizza and Pasta
        Currency: Euro
        """,
    },
    {
        "question": "What do you know about Greece?",
        "answer": """
        I know this:
        Capital: Athens
        Language: Greek
        Food: Souvlaki and Feta Cheese
        Currency: Euro
        """,
    },
]

example_prompt = PromptTemplate.from_template("Human: {question}\nAI:{answer}")

prompt = FewShotPromptTemplate(
    examples = examples,
    example_prompt = example_prompt,
    suffix = "Human: What do you know about {country}?",
    input_variables = ["country"],
)

# print(prompt)

chain = prompt | chat

chain.invoke({"country": "Spain"})

input_variables=['country'] input_types={} partial_variables={} examples=[{'question': 'What do you know about France?', 'answer': '\n        Here is what I know:\n        Capital: Paris\n        Language: French\n        Food: Wine and Cheese\n        Currency: Euro\n        '}, {'question': 'What do you know about Italy?', 'answer': '\n        I know this:\n        Capital: Rome\n        Language: Italian\n        Food: Pizza and Pasta\n        Currency: Euro\n        '}, {'question': 'What do you know about Greece?', 'answer': '\n        I know this:\n        Capital: Athens\n        Language: Greek\n        Food: Souvlaki and Feta Cheese\n        Currency: Euro\n        '}] example_prompt=PromptTemplate(input_variables=['answer', 'question'], input_types={}, partial_variables={}, template='Human: {question}\nAI:{answer}') suffix='Human: What do you know about {country}?'


## FewShotChatMessagePromptTemplate

In [11]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts.few_shot import FewShotChatMessagePromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts import ChatPromptTemplate, ChatMessagePromptTemplate

chat = ChatOpenAI(
    model = "gpt-4.1-nano",
    temperature = 0.1,
    streaming = True,
    callbacks = [
        StreamingStdOutCallbackHandler()
    ],
)

examples = [
    {
        "country": "France",
        "answer": """
        Here is what I know:
        Capital: Paris
        Language: French
        Food: Wine and Cheese
        Currency: Euro
        """,
    },
    {
        "country": "Italy",
        "answer": """
        I know this:
        Capital: Rome
        Language: Italian
        Food: Pizza and Pasta
        Currency: Euro
        """,
    },
    {
        "country": "Greece",
        "answer": """
        I know this:
        Capital: Athens
        Language: Greek
        Food: Souvlaki and Feta Cheese
        Currency: Euro
        """,
    },
]



In [ ]:
example_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "what do you know about {country}?"),
        ("ai", "{answer}")
    ]
)

example_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples
)

final_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a geography expert, you give short answer."),
        example_prompt,
        ("human", "What do you know about {country}?")
    ]
)

examples=[{'question': 'What do you know about France?', 'answer': '\n        Here is what I know:\n        Capital: Paris\n        Language: French\n        Food: Wine and Cheese\n        Currency: Euro\n        '}, {'question': 'What do you know about Italy?', 'answer': '\n        I know this:\n        Capital: Rome\n        Language: Italian\n        Food: Pizza and Pasta\n        Currency: Euro\n        '}, {'question': 'What do you know about Greece?', 'answer': '\n        I know this:\n        Capital: Athens\n        Language: Greek\n        Food: Souvlaki and Feta Cheese\n        Currency: Euro\n        '}] input_variables=[] input_types={} partial_variables={} example_prompt=ChatPromptTemplate(input_variables=['answer', 'country'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['country'], input_types={}, partial_variables={}, template='what do you know about {country}?'), additional_kwargs={}), AIMessagePrompt

In [13]:
chain = final_prompt | chat
chain.invoke(
    {
        "country" : "Korea"
    }
)


        Korea is divided into two countries:
        North Korea (Pyongyang) and South Korea (Seoul).
        Languages: Korean.
        South Korea is known for technology and pop culture; North Korea is more isolated.
        Currency: South Korean Won; North Korean Won.

AIMessage(content='\n        Korea is divided into two countries:\n        North Korea (Pyongyang) and South Korea (Seoul).\n        Languages: Korean.\n        South Korea is known for technology and pop culture; North Korea is more isolated.\n        Currency: South Korean Won; North Korean Won.', additional_kwargs={}, response_metadata={'finish_reason': 'stop'}, id='run--68cb3c67-38f7-4f9a-bd51-7cb60041fbef-0')

# RandomBasedExampleSelector

In [14]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import example_selector
from langchain.prompts.few_shot import FewShotPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts.prompt import PromptTemplate
from langchain.prompts.example_selector.base import BaseExampleSelector

chat = ChatOpenAI(
    model = "gpt-4.1-nano",
    temperature = 0.1,
    streaming = True,
    callbacks = [
        StreamingStdOutCallbackHandler()
    ]
)

examples = [
    {
        "question": "What do you know about France?",
        "answer": """
        Here is what I know:
        Capital: Paris
        Language: French
        Food: Wine and Cheese
        Currency: Euro
        """,
    },
    {
        "question": "What do you know about Italy?",
        "answer": """
        I know this:
        Capital: Rome
        Language: Italian
        Food: Pizza and Pasta
        Currency: Euro
        """,
    },
    {
        "question": "What do you know about Greece?",
        "answer": """
        I know this:
        Capital: Athens
        Language: Greek
        Food: Souvlaki and Feta Cheese
        Currency: Euro
        """,
    },
]



In [15]:
class RandomExampleSelector(BaseExampleSelector):
    def __init__(self, examples):
        self.examples = examples

    def add_example(self, example):
        self.examples.append(examples)

    def select_examples(self, input_variables):
        from random import choice

        return [choice(self.examples)]

In [16]:
example_prompt = PromptTemplate.from_template("Human: {question}\nAI:{answer}")

example_selector = RandomExampleSelector(
    examples=examples
)

In [17]:
prompt = FewShotPromptTemplate(
    example_prompt = example_prompt,
    example_selector = example_selector,
    suffix = "Human: What do you know about {country}?",
    input_variables = ["country"]
)

prompt.format(country = "Brazil")

'Human: What do you know about France?\nAI:\n        Here is what I know:\n        Capital: Paris\n        Language: French\n        Food: Wine and Cheese\n        Currency: Euro\n        \n\nHuman: What do you know about Brazil?'